In [1]:
pip install scikit-learn nltk 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import yaml
from sqlalchemy import create_engine
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import numpy as np
import nltk

## Database Connection

In [3]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [4]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [5]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Load table mensajeria_novedadesservicio 

In [6]:
novedad_servicio = pd.read_sql_table("mensajeria_novedadesservicio",url_db)
novedad_servicio.head(5)

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7


In [7]:
num_clusters = 10  # Número original de grupos
extra_group_name = "Grupo Especial"  # Nombre para el nuevo grupo

# Definir una lista de stopwords en español
stopwords_spanish = [
    'yo', 'tú', 'él', 'ella', 'usted', 'nosotros', 'vosotros', 'ellos',
    'la', 'el', 'al','los', 'las', 'de', 'y', 'en', 'que', 'a', 'es', 'un', 
    'una', 'por', 'para', 'con', 'no', 'si', 'se', 'me', 'te', 'le', 
    'lo', 'la', 'sus', 'mi', 'su', 'esto', 'eso', 'aquello', 'este', 
    'esa', 'aquella', 'ser', 'haber', 'estar', 'tener', 'hacer', 
    'poder', 'decir', 'ir', 'ver', 'dar', 'saber', 'querer', 'llegar',
    'pasar', 'deber', 'poner', 'parecer', 'quedar', 'creer', 'hablar', 
    'llevar', 'dejar', 'seguir', 'encontrar', 'llamar', 'venir', 
    'pensar', 'salir', 'volver', 'tomar', 'conocer', 'vivir', 
    'sentir', 'tratar', 'mirar', 'contar', 'empezar', 'esperar',
    'buscar', 'existir', 'entrar', 'trabajar', 'escribir', 
    'perder', 'producir', 'ocurrir', 'entender', 'poder', 'probar',
    'saber', 'así', 'mientras', 'cualquier', 'algunos', 'muchos', 
    'todos', 'cada', 'pocos', 'nada', 'todo', 'b3', 'b66', 'ya', 'qué'
]

# Separar documentos especiales y normales
special_documents = []
normal_documents = []

for doc in novedad_servicio['descripcion']:
    # Verificar si el documento es una letra o una sola palabra
    if len(doc.strip()) == 1 or (len(doc.strip().split()) == 1):
        special_documents.append(doc)
    else:
        normal_documents.append(doc)

# Combinar los documentos normales y especiales
documents = normal_documents + special_documents

# Crear el vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words=stopwords_spanish)
X = vectorizer.fit_transform(documents)

# Instanciar y ajustar el modelo K-Means
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Obtener los nombres de las características (términos) de la matriz TF-IDF
terms = vectorizer.get_feature_names_out()

# Crear un diccionario para almacenar las palabras clave de cada grupo
top_terms_per_cluster = {}

# Obtener las etiquetas de los clusters
labels = kmeans.labels_

# Agrupar los términos por clusters
for i in range(num_clusters):
    # Obtener los índices de los puntos en el grupo i
    cluster_indices = np.where(labels == i)[0]
    # Obtener las filas correspondientes en la matriz TF-IDF
    cluster_tfidf = X[cluster_indices]
    # Calcular el promedio de peso de cada término dentro del grupo
    term_weights = np.mean(cluster_tfidf.toarray(), axis=0)
    # Obtener los términos con el peso más alto en el grupo
    top_terms_indices = term_weights.argsort()[-10:][::-1]  # Top 8 términos
    top_terms = [terms[index] for index in top_terms_indices]
    # Almacenar los términos para el grupo
    top_terms_per_cluster[f"Grupo {i}"] = top_terms

# Agregar el grupo especial
top_terms_per_cluster[extra_group_name] = special_documents

# Mostrar los términos clave de cada grupo
top_terms_per_cluster


{'Grupo 0': ['está',
  'informa',
  'listo',
  'cerrado',
  'servicio',
  'asesor',
  'llamando',
  'bodega',
  'cliente',
  'repuesto'],
 'Grupo 1': ['espera',
  'están',
  'sigo',
  'entreguen',
  'terminal',
  'reciban',
  'latín',
  'rod',
  'masivo',
  'encomienda'],
 'Grupo 2': ['pedido',
  'despachen',
  'esperando',
  'espera',
  'entreguen',
  'despacho',
  'filtros',
  'redox',
  'b1',
  'bodega'],
 'Grupo 3': ['toma',
  'compañero',
  'otro',
  'asunto',
  'trasbordo',
  'servicio',
  'vicuña',
  'muñoz',
  'jairo',
  'julian'],
 'Grupo 4': ['servicio',
  'camioneta',
  'mal',
  'programado',
  'persona',
  'tiene',
  'abandono',
  'llegado',
  'cancelaron',
  'mañana'],
 'Grupo 5': ['compañero',
  'otro',
  'servicio',
  'entrego',
  'continúa',
  'entrega',
  'luis',
  'hace',
  'cardona',
  'cabal'],
 'Grupo 6': ['enterado',
  'pendiente',
  'ok',
  'recoger',
  'salgo',
  'asesor',
  'b20',
  'lópez',
  'estoy',
  'jeferson'],
 'Grupo 7': ['deja',
  'repuesto',
  'escrit

In [8]:
novedad_servicio.head(5)

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7


In [9]:
# Diccionario con descripciones de cada grupo
descriptions = {
    'Grupo 0': 'Estado del servicio y atención al cliente.',
    'Grupo 1': 'Coordinación de entregas y tiempos de espera.',
    'Grupo 2': 'Gestión de pedidos y despachos.',
    'Grupo 3': 'Interacción y colaboración entre compañeros.',
    'Grupo 4': 'Problemas y situaciones adversas en servicios.',
    'Grupo 5': 'Trabajo en equipo y continuidad en entregas.',
    'Grupo 6': 'Seguimiento de acciones pendientes y recogida.',
    'Grupo 7': 'Logística de repuestos y tareas administrativas.',
    'Grupo 8': 'Estado de alistamiento y procesos de entrega.',
    'Grupo 9': 'Disponibilidad de productos y comunicación al cliente.',
    "Grupo Especial": "Descripciones sin Sentido"
}

# Convertir el diccionario a un DataFrame
categoria_descripcion = pd.DataFrame([
    {
        "grupo": grupo,
        "terminos_clave": ", ".join(terminos),
        "descripcion": descriptions[grupo]
    }
    for grupo, terminos in top_terms_per_cluster.items()
])
categoria_descripcion

,grupo,terminos_clave,descripcion
0,Grupo 0,"está, informa, listo, cerrado, servicio, aseso...",Estado del servicio y atención al cliente.
1,Grupo 1,"espera, están, sigo, entreguen, terminal, reci...",Coordinación de entregas y tiempos de espera.
2,Grupo 2,"pedido, despachen, esperando, espera, entregue...",Gestión de pedidos y despachos.
3,Grupo 3,"toma, compañero, otro, asunto, trasbordo, serv...",Interacción y colaboración entre compañeros.
4,Grupo 4,"servicio, camioneta, mal, programado, persona,...",Problemas y situaciones adversas en servicios.
5,Grupo 5,"compañero, otro, servicio, entrego, continúa, ...",Trabajo en equipo y continuidad en entregas.
6,Grupo 6,"enterado, pendiente, ok, recoger, salgo, aseso...",Seguimiento de acciones pendientes y recogida.
7,Grupo 7,"deja, repuesto, escritorio, yulieth, mañana, s...",Logística de repuestos y tareas administrativas.
8,Grupo 8,"esperando, despacho, bodega, entrega, reciban,...",Estado de alistamiento y procesos de entrega.
9,Grupo 9,"repuesto, entrega, hay, informa, encuentra, se...",Disponibilidad de productos y comunicación al ...


## Load table into ETL database

In [10]:
categoria_descripcion.to_sql("dim_descripcion_novedad", etl_conn, index_label="key_descripcion_novedad", if_exists="replace") 

11